In [7]:
# import the necessary packages
import numpy as np
import argparse
import time
import cv2
import os
import glob
confidence1=.1


# load the COCO class labels our YOLO model was trained on
labelsPath = "F:\\yolo_weights\\obj.names"
LABELS = open(labelsPath).read().strip().split("\n")
threshold1=.9
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
 dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath ="F:\\yolo_weights\\yolo-obj_28000.weights"
configPath = "F:\\yolo_weights\\yolo-obj.cfg"

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# load our input image and grab its spatial dimensions
filenames=glob.glob("F:\\thumbnails1"+'/*')
# print(filenames)
print(len(filenames))

[INFO] loading YOLO from disk...
238


In [8]:
j=0
for i in filenames:
    try:
        image = cv2.imread(i)
        (H, W) = image.shape[:2]

        # determine only the *output* layer names that we need from YOLO
        ln = net.getLayerNames()
        ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

        # construct a blob from the input image and then perform a forward
        # pass of the YOLO object detector, giving us our bounding boxes and
        # associated probabilities
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
            swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(ln)
        end = time.time()
        # show timing information on YOLO
        print("[INFO] YOLO took {:.6f} seconds".format(end - start))

        # initialize our lists of detected bounding boxes, confidences, and
        # class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []
#         print(i)
        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability) of
                # the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
#                 print(classID)
                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence >= confidence1:
                    # scale the bounding box coordinates back relative to the
                    # size of the image, keeping in mind that YOLO actually
                    # returns the center (x, y)-coordinates of the bounding
                    # box followed by the boxes' width and height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")

                    # use the center (x, y)-coordinates to derive the top and
                    # and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # update our list of bounding box coordinates, confidences,
                    # and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

#         for ind, i in enumerate(boxes):
#             print(i, confidences[ind], classIDs[ind])
        # apply non-maxima suppression to suppress weak, overlapping bounding
        # boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence1,
            threshold1)

        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                # draw a bounding box rectangle and label on the image
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
                j=j+1
        # show the output image
#         cv2.imwrite("E:\\9conf\\"+str(j) +".jpeg",image)
#         j=j+1
    except: 
        continue
print(j)

[INFO] YOLO took 1.201578 seconds
[INFO] YOLO took 0.547534 seconds
[INFO] YOLO took 0.486745 seconds
[INFO] YOLO took 0.480196 seconds
[INFO] YOLO took 0.447541 seconds
[INFO] YOLO took 0.446133 seconds
[INFO] YOLO took 0.455821 seconds
[INFO] YOLO took 0.533990 seconds
[INFO] YOLO took 0.598101 seconds
[INFO] YOLO took 0.535268 seconds
[INFO] YOLO took 0.473353 seconds
[INFO] YOLO took 0.451750 seconds
[INFO] YOLO took 0.471199 seconds
[INFO] YOLO took 0.491734 seconds
[INFO] YOLO took 0.469359 seconds
[INFO] YOLO took 0.462821 seconds
[INFO] YOLO took 0.477161 seconds
[INFO] YOLO took 0.463967 seconds
[INFO] YOLO took 0.494368 seconds
[INFO] YOLO took 0.466641 seconds
[INFO] YOLO took 0.481263 seconds
[INFO] YOLO took 0.454390 seconds
[INFO] YOLO took 0.483679 seconds
[INFO] YOLO took 0.449467 seconds
[INFO] YOLO took 0.449311 seconds
[INFO] YOLO took 0.438498 seconds
[INFO] YOLO took 0.450522 seconds
[INFO] YOLO took 0.453758 seconds
[INFO] YOLO took 0.470652 seconds
[INFO] YOLO to

In [9]:
print(j)

161
